In [23]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

In [1]:
from __future__ import annotations
import os

import ee
import google.auth

import folium
from serving import data
import json
from branca.element import Figure
from folium import plugins
import sys
from serving.data import get_input_image
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from importlib import reload
import pandas as pd

In [2]:
current_dir = os.getcwd()
sys.path.append(current_dir+'/serving')

In [3]:
# Google cloud parameters
project = "supple-nature-370421"  # @param {type:"string"}
bucket = "vgnn"  # @param {type:"string"}
location = "us-west2-a"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

# Initialise goofle earth engine
credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

Updated property [core/project].


In [4]:
#CONSTANTS
YEAR_START=2017
YEAR_END=2023

SEASON_START=5
SEASON_END=10

CROP=1 #Corn

# True vision params
vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],}

In [5]:
# reload(data)

# Show sample filter

start_year = 2017
start_month = 4
end_month = 7
crop_type = 1 # Corn
county = "Lancaster"

county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("NAME", county))
).geometry()

coords = county_geom.centroid().coordinates().getInfo()

s2_img_start = get_input_image(county, crop_type, start_year, start_month)["image"]
s2_img_end = get_input_image(county, crop_type, start_year, end_month)["image"]

In [7]:
# Create the map
m1 = folium.Map(coords[::-1])

# Add the CDL layer
layer_left = folium.TileLayer(
    tiles=s2_img_start.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True
)

layer_right = folium.TileLayer(
    tiles=s2_img_end.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True,
)

sbs = plugins.SideBySideLayers(layer_left=layer_left, layer_right=layer_right)

layer_left.add_to(m1)
layer_right.add_to(m1)
sbs.add_to(m1)    

# Display the map
m1

In [8]:
"""
Changes in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power
"""

'\nChanges in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power\n'

In [55]:
img_name = f"{county}_{start_year}_{end_month}-{end_month+1}"
img_task = ee.batch.Export.image.toCloudStorage(
    image=s2_img_end,
    description=img_name,
    bucket=bucket,
    fileNamePrefix=f"images/{img_name}",
    region=county_geom,
    scale=100,
    skipEmptyTiles = True,
    fileFormat = "GeoTIFF",
    maxPixels = 1e9
)
img_task.start()

In [65]:
counties = ["Lancaster", "Crawford", "Orleans", "Tulare"]#, "Wharton", "Story", "Canyon", "Kit Carson"]
# years = range(2018,2023)
years = range(2022,2023)
months = [9]
scales = [50 * (1+x) for x in range(len(counties))]

get_input_img_params = [{"county": county,
                         "crop": CROP,
                        "year": year,
                        "month": month} for county in counties
                                       for year in years
                                       for month in months
                       ]


In [26]:
# [{'county': 'Lancaster', 'crop': 1, 'year': 2022, 'month': 9}]
#     img_task = ee.batch.Export.image.toCloudStorage(
#     image=image,
#     description=image_name,
#     bucket=bucket,
#     fileNamePrefix=f"images/{image_name}",
#     scale=150,
#     region=county_geom,
#     # fileDimensions = 2048,
#     skipEmptyTiles = True,
#     fileFormat = "GeoTIFF",
#     maxPixels = 1e10

# [{'county': 'Story', 'crop': 1, 'year': 2020, 'month': 9}]
#     img_task = ee.batch.Export.image.toCloudStorage(
#     image=image,
#     description=image_name,
#     bucket=bucket,
#     fileNamePrefix=f"images/{image_name}",
#     scale=150,
#     region=county_geom,
#     fileDimensions = 2*2048,
#     skipEmptyTiles = True,
#     fileFormat = "GeoTIFF",
#     maxPixels = 1e9

[{'county': 'Story', 'crop': 1, 'year': 2020, 'month': 9}]

In [72]:
def export_img(image:ee.Image, image_name: str, county_geom: ee.geometry):
    
    scale = 200
    image_name += f"_{scale}"
    img_task = ee.batch.Export.image.toCloudStorage(
    image=image,
    description=image_name,
    bucket=bucket,
    fileNamePrefix=f"images/{image_name}",
    scale=scale,
    region=county_geom,
    # fileDimensions = 4*2048,
    skipEmptyTiles = True,
    fileFormat = "GeoTIFF",
    maxPixels = 1e9
    )

    img_task.start()

def apply_get_input_image(params):
    return get_input_image(**params)

def unpack_for_export(list_input):
    county = list_input["image_name"].split("_")[0]
    county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("NAME", county))
        ).geometry()
    
    return export_img(list_input["image"], list_input["image_name"], county_geom)

In [71]:
with beam.Pipeline() as pipeline:
    (
        pipeline
        |"Create parameter sets" >>  beam.Create(get_input_img_params)
        | "Sample counties" >> beam.Map(apply_get_input_image)
        | "Export image to GCS bucket" >> beam.Map(unpack_for_export)
    )

In [67]:
get_input_img_params

[{'county': 'Lancaster', 'crop': 1, 'year': 2022, 'month': 9},
 {'county': 'Crawford', 'crop': 1, 'year': 2022, 'month': 9},
 {'county': 'Orleans', 'crop': 1, 'year': 2022, 'month': 9},
 {'county': 'Tulare', 'crop': 1, 'year': 2022, 'month': 9}]